In [1]:
!python3 --version

Python 3.10.12


#### fix for could not import -lmza
https://github.com/lucidrains/imagen-pytorch/issues/92 

#### Fix for Can't import datasets AttributeError: partially initialized module 'datasets' has no attribute 'utils' (most likely due to a circular import)

Restart juypyter and kernel


In [2]:
import warnings
warnings.filterwarnings('ignore')


In [3]:
import helper
import time
helper.check_gpu_capacity()

GPU Information:
GPU 0:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB
GPU 1:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB
GPU 2:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB
GPU 3:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB


In [4]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

print(f"dataset size: {len(dataset)}")

# dataset size: 15011


Found cached dataset json (/home/ubuntu/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


dataset size: 15011


In [5]:
sample_ds = dataset[randrange(len(dataset))]

for key, value in sample_ds.items():
    print(f"--> {key}")
    print("====================")
    print(value)
    print("====================")

--> instruction
What are the interests behind climate change denial?
--> context

--> response
Financial interest from industries opposed to the regulation of CO2 emissions. Migrating to low carbon technologies would require massive investments, making executives concerned about impacts on companies gross margin.
--> category
open_qa


In [6]:
set(dataset['category'])

{'brainstorming',
 'classification',
 'closed_qa',
 'creative_writing',
 'general_qa',
 'information_extraction',
 'open_qa',
 'summarization'}

In [7]:
# len(dataset.map( ['context']))

In [8]:
# list(filter(lambda x :x != '', list(map(lambda x:((x['category'], len(x['context'])) if len(x['context']) > 5000 else ''), dataset))))

Based on our use case, we want to focus on 'general_qa' and  'information_extraction tasks.,

In [9]:
target_categories = ['closed_qa', 'summarization', 'information_extraction']
filtered_dataset = dataset.filter(lambda x: x['category'] in target_categories)

print(set(filtered_dataset['category']))
filtered_dataset

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-67fe253a0190166e.arrow


{'information_extraction', 'summarization', 'closed_qa'}


Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 4467
})

In [10]:
sample_ds = filtered_dataset[randrange(len(filtered_dataset))]

for key, value in sample_ds.items():
    print(f"--> {key}")
    print("====================")
    print(value)
    print("====================")

--> instruction
Given this paragraph about a Tree well, why is it so dangerous for hikers, snowboarder, skiers and any snow practitioners
--> context
A tree well, also known as a spruce trap, is the space around a tree under its branches that does not get the same amount of snow as the surrounding open space. This creates a void or area of loose snow below the branches and around the trunk that is dangerous to any hikers, snowshoers, skiers, snowboarders, and snowmobilers who fall into them. If someone lands in such a well, often as a result of a fall, it can be too deep for them to climb up the surrounding loose snow before they are buried. Making the situation more dangerous, they often fall into the well head-first and as the result of an accident which could leave them injured or unconscious
--> response
It is dangerous because it creates an area of loose snow below the branches and around the trunk, and when someone will fall into this area it can be too deep for this person to cl

To instruct tune our model, we need to convert our structured examples into a collection of tasks described via instructions. We define a formatting_function that takes a sample and returns a string with our format instruction.

In [11]:
def format_instruction(sample):
	return f"""### Instruction:
            Use the Input below to create an instruction, which could have been used to generate the input using an LLM.
            
            ### Input:
            {sample['context'] if sample['context'] != '' else ''}

            {sample['response']}

            ### Response:
            {sample['instruction']}
        """


Let's test our formatting function on a random example.

In [12]:
from random import randrange

print(format_instruction(filtered_dataset[randrange(len(filtered_dataset))]))

## Do I need to add the context in the format_instruction?????

### Instruction:
            Use the Input below to create an instruction, which could have been used to generate the input using an LLM.
            
            ### Input:
            Michel Warschawski (Mikado) (born 25 July 1949) is an Israeli anti-Zionist activist. He led the Marxist Revolutionary Communist League (previously Matzpen-Jerusalem) until its demise in the 1990s, and founded the Alternative Information Center, a joint Palestinian-Israeli non-governmental organization, in 1984.

            Michel Warschawski, an anti-Zionist activist, led the Marxist Revolutionary Communist League and later the Alternative Information Center.

            ### Response:
            Given this paragraph about Michel Warschawski, tell me the two organizations that Warschawski led during his life.
        


In [13]:
print(format_instruction(sample_ds))

### Instruction:
            Use the Input below to create an instruction, which could have been used to generate the input using an LLM.
            
            ### Input:
            A tree well, also known as a spruce trap, is the space around a tree under its branches that does not get the same amount of snow as the surrounding open space. This creates a void or area of loose snow below the branches and around the trunk that is dangerous to any hikers, snowshoers, skiers, snowboarders, and snowmobilers who fall into them. If someone lands in such a well, often as a result of a fall, it can be too deep for them to climb up the surrounding loose snow before they are buried. Making the situation more dangerous, they often fall into the well head-first and as the result of an accident which could leave them injured or unconscious

            It is dangerous because it creates an area of loose snow below the branches and around the trunk, and when someone will fall into this area it c

In [14]:
shuffled_dataset = dataset.shuffle(seed=42)
shuffled_dataset = shuffled_dataset.train_test_split(test_size=0.1)

train_dataset = shuffled_dataset['train']
test_dataset = shuffled_dataset['test']

train_eval_dataset = train_dataset.train_test_split(test_size=0.2)
train_dataset = train_eval_dataset['train']
eval_dataset = train_eval_dataset['test']


# Print the sizes of the train, eval, andtest sets
print("Dataset size:", len(dataset))
print("Train size:", len(train_dataset))
print("Eval size:", len(eval_dataset))
print("Test size:", len(test_dataset))

Loading cached shuffled indices for dataset at /home/ubuntu/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-645f708b2cbd5333.arrow


Dataset size: 15011
Train size: 10807
Eval size: 2702
Test size: 1502


### Instruction-tune Llama 2 using trl and the SFTTrainer

In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, \
AutoModelForSeq2SeqLM, BitsAndBytesConfig, Trainer, T5Tokenizer, T5ForConditionalGeneration


use_flash_attention = False
# COMMENT IN TO USE FLASH ATTENTION
# replace attention with flash attention
# if torch.cuda.get_device_capability()[0] >= 8:
#     from utils.llama_patch import replace_attn_with_flash_attn
#     print("Using flash attention")
#     replace_attn_with_flash_attn()
#     use_flash_attention = True

helper.check_gpu_capacity()


GPU Information:
GPU 0:
  Total Memory: 23028 MB
  Free Memory: 22508 MB
  Used Memory: 7 MB
GPU 1:
  Total Memory: 23028 MB
  Free Memory: 22508 MB
  Used Memory: 7 MB
GPU 2:
  Total Memory: 23028 MB
  Free Memory: 22508 MB
  Used Memory: 7 MB
GPU 3:
  Total Memory: 23028 MB
  Free Memory: 22508 MB
  Used Memory: 7 MB


In [16]:
# # Hugging Face model id
# # model_flant5_base_id = "NousResearch/Llama-2-7b-hf" # non-gated
# # model_id = "meta-llama/Llama-2-7b-hf" # gated

# # BitsAndBytesConfig int-4 config
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# # Load model and tokenizer
# model_llama_2_7b = AutoModelForCausalLM.from_pretrained(model_llama_2_7b_id, 
#                                                         quantization_config=bnb_config, 
#                                                         use_cache=False, 
#                                                         device_map="auto")
# tokenizer_llama_2_7b = AutoTokenizer.from_pretrained(model_llama_2_7b_id)

In [17]:
!pip install sentencepiece -q
!pip install protobuf -q

In [39]:
# Hugging Face model id
model_flan_t5_base_id = "lmsys/vicuna-7b-v1.5-16k" 

model_flan_t5_base_id = "lmsys/vicuna-7b-v1.5"


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
torch.cuda.set_device(2)

# Load model and tokenizer
model_flan_t5_base = AutoModelForCausalLM.from_pretrained(model_flan_t5_base_id, 
                                                        quantization_config=bnb_config, 
                                                        use_cache=False, 
                                                        device_map="auto",
                                                          # device = 'cuda:2',
                                                        torch_dtype=torch.bfloat16
                                                          )

# # model_flan_t5_base.to(f'cuda:1')
# tokenizer_flan_t5_base = AutoTokenizer.from_pretrained(model_flan_t5_base_id)
# model_flan_t5_base.hf_device_map



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [40]:
tokenizer_flan_t5_base = AutoTokenizer.from_pretrained(model_flan_t5_base_id)

In [41]:
## From above, we see that the generated response is not okay

In [42]:

model_flan_t5_base.config.pretraining_tp = 1

tokenizer_flan_t5_base.pad_token = tokenizer_flan_t5_base.eos_token
tokenizer_flan_t5_base.padding_side = "right"

In [22]:
prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
{sample_ds['context']}

{sample_ds['response']}

### Response:
"""

input_ids = tokenizer_flan_t5_base(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model_flan_t5_base.generate(input_ids=input_ids, max_new_tokens=1000, do_sample=True, top_p=0.9,temperature=0.9)

print(f"Prompt:\n{sample_ds['response']}\n")
print(f"Generated instruction:\n{tokenizer_flan_t5_base.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"Ground truth:\n{sample_ds['instruction']}")


Prompt:
It is dangerous because it creates an area of loose snow below the branches and around the trunk, and when someone will fall into this area it can be too deep for this person to climb up before they are buried. Another dangerous aspect is the fact that they could fall head-first which could leave them injured or unconscious.

Generated instruction:

Hikers, snowshoers, skiers, snowboarders, and snowmobilers should be aware of the potential danger of tree wells, also known as spruce traps. These areas can be hazardous as they are prone to creating an area of loose snow below the branches and around the trunk, which can be deep enough to bury someone if they fall into it. It's important to be cautious while navigating in and around trees, especially in areas with a high risk of snow accumulation, as a fall could result in a head-first descent into the well, potentially leading to injury or unconsciousness.
Ground truth:
Given this paragraph about a Tree well, why is it so dangero

The SFTTrainer supports a native integration with peft, which makes it super easy to efficiently instruction tune LLMs. We only need to create our LoRAConfig and provide it to the trainer.

In [23]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"



In [43]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)

# prepare model for training
model_flan_t5_base = prepare_model_for_kbit_training(model_flan_t5_base)
model_flan_t5_base = get_peft_model(model_flan_t5_base, peft_config)

# print(print_number_of_trainable_model_parameters(model_flan_t5_base_peft))

model_flan_t5_base.print_trainable_parameters()

trainable params: 33,554,432 || all params: 3,533,967,360 || trainable%: 0.9494833591219133


Before we can start our training we need to define the hyperparameters (TrainingArguments) we want to use.

In [44]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="models/vicuna-7b-instruct-dolly-filtered",
    num_train_epochs=4,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    # save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=False, # disable tqdm since with packing values are in correct,
    save_total_limit = 2,
    save_strategy = "no",
    load_best_model_at_end=False
)


In [45]:
tokenizer_flan_t5_base.model_max_length

4096

In [46]:
from trl import SFTTrainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model_flan_t5_base,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    max_seq_length=tokenizer_flan_t5_base.model_max_length,
    tokenizer=tokenizer_flan_t5_base,
    packing=True,
    formatting_func=format_instruction,
    args=args,
)


In [49]:
helper.check_gpu_capacity()
# del model_llama_2_7b
# del model_llama_2_7b_kbit
helper.free_gpu_memory()
helper.memory_stats()


GPU Information:
GPU 0:
  Total Memory: 23028 MB
  Free Memory: 3869 MB
  Used Memory: 18646 MB
GPU 1:
  Total Memory: 23028 MB
  Free Memory: 3061 MB
  Used Memory: 19454 MB
GPU 2:
  Total Memory: 23028 MB
  Free Memory: 1003 MB
  Used Memory: 21512 MB
GPU 3:
  Total Memory: 23028 MB
  Free Memory: 8837 MB
  Used Memory: 13678 MB
GPU Memory Used: 2.43 GB
GPU Memory Cached: 5.44 GB


In [48]:
%%time

# train
trainer.train()



Token indices sequence length is longer than the specified maximum sequence length for this model (5418 > 4096). Running this sequence through the model will result in indexing errors
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 GiB (GPU 2; 21.99 GiB total capacity; 6.73 GiB already allocated; 1003.75 MiB free; 9.78 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [49]:
# save model
trainer.save_model()

## Test Model and run Inference

In [30]:
import torch

from peft import AutoPeftModelForCausalLM, AutoPeftModelForSeq2SeqLM, PeftModel, PeftConfig, AutoPeftModel
from transformers import AutoTokenizer

# args.output_dir = "model/flan-t5-xl-instruct-dolly-filtered/checkpoint-1791"

# load base LLM model and tokenizer
# model = AutoPeftModelForCausalLM.from_pretrained(
#     args.output_dir,
#     low_cpu_mem_usage=True,
#     torch_dtype=torch.float16,
#     load_in_4bit=True,
# )
# tokenizer = AutoTokenizer.from_pretrained(args.output_dir)


# model = AutoPeftModelForSeq2SeqLM.from_pretrained(
#     args.output_dir,
#     # peft_training_args.output_dir,
#     torch_dtype=torch.float16,
#     is_trainable=False
# )

# load base LLM model and tokenizer
model = T5ForConditionalGeneration.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
# tokenizer = AutoTokenizer.from_pretrained(args.output_dir)

tokenizer = T5Tokenizer.from_pretrained(args.output_dir)

# model = T5ForConditionalGeneration.from_pretrained(
#         args.output_dir,
#         # low_cpu_mem_usage=True,
#         # torch_dtype=torch.float16,
#         # load_in_4bit=True,
#         device_map="auto"
# )





OSError: models/flan-t5-xl-instruct-dolly-filtered does not appear to have a file named config.json. Checkout 'https://huggingface.co/models/flan-t5-xl-instruct-dolly-filtered/main' for available files.

In [ ]:
input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))


Let’s load the dataset again with a random sample to try to generate an instruction.

In [85]:
# from datasets import load_dataset
# from random import randrange


# Load dataset from the hub and get a sample
# dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
sample = dataset[randrange(len(dataset))]

prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
{sample_ds['response']}

### Response:
"""

# prompt = f"""### Instruction:
# Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

# ### Input:
# 4.1.	UV-Light
# Together with oxygen, UV-light can produce a singlet oxygen which can form directly a hydroperoxide from a unsaturated lipid (see Figure 3). Usually during the processing of powders, prevention of UV-light intrusion is being controlled. Furthermore, it depends on the package used, but as well how the consumer stores the product. 
# 4.2.	Oxygen
# Reaction with oxygen can take at various sites in the lipid oxidation chain reaction (see Figure 3). Exclusion of oxygen means mainly flushing inert gases typically nitrogen (or argon) to displace oxygen, packaging in vacuum or in modified atmosphere packaging (MAP). Oxygen scavengers can be employed to completely remove residual oxygen from the system. 
# 4.3.	Heat
# Higher temperatures speed up the lipid oxidation reaction rates. Although the product should be prevented from heat, during production heat is needed to: 
# 1.	be able to handle ingredients (e.g. melting of fat, best done under nitrogen)
# 2.	get a product microbiologically stable
# 3.	spray dry the product
# Clearly heat is needed during production but where possible it should be minimized to prevent lipid oxidation. 

# ### Response:
# """

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.9)

print('GGG\n', tokenizer.decode(outputs[0], skip_special_tokens=True))

# print(f"Prompt:\n{sample_ds['response']}\n")
# print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"Ground truth:\n{sample_ds['instruction']}")


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [84]:
sample_ds['response']

'Landscaping is a blend of art and science. In order to be a proficient landscaper, one must scientifically understand which types of plants react to which types of elements and conditions. In order to create a thriving landscape, the science must make sense amongst these elements for the ecosystem to thrive. Landscaping is also artistic in the sense of creating something that fits into the surroundings and is pleasing to the eye. The best landscapers understand the science and also have an artistic ability to express beauty.'

### Evaluate the Model Quantitatively (with ROUGE Metric)
Perform inferences for the sample of the test dataset (only 50 instructions and responses to save time). 

In [34]:
dataset

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 15011
})

In [35]:
import pandas as pd


instruction_test = dataset['instruction'][50:100]
response_test = dataset['response'][50:100]

generated_instruction_list = []
ground_truth_list = []

for instruction, response in zip(instruction_test, response_test):
    prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
{response}

### Response:
"""

    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
    # with torch.inference_mode():
    outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.2)
    
    # print(f"Prompt:\n{sample_ds['response']}\n")
    generated_instruction = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]
    generated_instruction = generated_instruction.split('\n')[0].strip()

    generated_instruction_list.append(generated_instruction)
    ground_truth_list.append(instruction)

zipped_summaries = list(zip(generated_instruction_list, ground_truth_list))
 
df = pd.DataFrame(zipped_summaries, columns = ['generated_instruction_list', 'ground_truth_list'])
df

,generated_instruction_list,ground_truth_list
0,What are the 10 commandments of life?,Think of some family rules to promote a health...
1,Who is Karlon Stark,"In the series A Song of Ice and Fire, who is t..."
2,What is enriched air?,What is enriched air and why would divers dive...
3,What is Casablanca?,What film won the 1943 Oscar as best film
4,Is cold water heavier than hot water?,"which weighs more, cold or hot water?"
5,What is the top speed of a Kia Stinger?,"Given this paragraph, what is the top speed of..."
6,Is it a good idea to have both a cat and a bir...,Write a short paragraph about why you should n...
7,What is the publishing rate of individual stri...,What is your favorite strip from the comic Cal...
8,What are the main aspects that make F1 cars fast?,What makes a formula one car so fast?
9,What is the Key lime pie?,Without quoting directly from the text give me...


In [37]:
for index, row in df[40:50].iterrows():
    print('GI: ', row['generated_instruction_list'])

    print('GT: ',row['ground_truth_list'], '\n')

GI:  What is the difference between Viola toeria and Samphor?
GT:  Identify which instrument is string or percussion: Samphor, Viola toeria 

GI:  What are the things found inside a house?
GT:  If we were playing a game where we had to identify things that can be found inside a house, which of these would we call out: car, chair, table, park, cloud, microwave. 

GI:  What are the different types of beer?
GT:  Classify the following as either dark-colored beers or light colored beers: porter, pilsner, stout, amber, lager 

GI:  
GT:  Give me a bulleted list of the 5 highest mountains in the world and their respective heights in meters 

GI:  • When was Daniel Leavitt born?
GT:  Extract all of the dates mentioned in this paragraph and list them using bullets in the format {Date} - {Description} 

GI:  What products does Apple sell?
GT:  Which products apple sell? 

GI:  What is the moral of the story?
GT:  Write a short story about a person who discovers a hidden room in their house. The

In [38]:
!pip install evaluate==0.4.0 rouge_score==0.1.2 --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Compute ROUGE score for this subset of the data. 

In [39]:
import evaluate

rouge = evaluate.load('rouge')

model_results = rouge.compute(
    predictions=generated_instruction_list,
    references=ground_truth_list[0:len(generated_instruction_list)],
    use_aggregator=True,
    use_stemmer=True,
)

print('MODEL:')
model_results_df = pd.DataFrame(model_results.items(), columns=['Metric', 'Value'])
model_results_df

MODEL:


,Metric,Value
0,rouge1,0.467613
1,rouge2,0.297052
2,rougeL,0.436774
3,rougeLsum,0.434922
